# MINIST XGBOOST 

This is a model with 96% accuracy, after solving the following issue , it should be better

####  Crash after setting ntree_limit= model.best_ntree_limit, correct it with:
model.set_param('predictor','cpu_predictor')



1. __Business Understanding__: Using MINIST dataset to train model , then recognize digit image
2. __Analytic Approach:__ XGBOOST
3. __Data Requirement:__ using minist data [Kaggle digit recognizer](https://www.kaggle.com/c/digit-recognizer/data)
4. __Data Collection:__ using pandas, read from csv file
5. __Data Understanding:__ using pandas and visulizatio seaborn, it's easy, no complex featurs, only pixels(724)
6. __Data Preparation:__ pandas , no need more effort 
7. __Modeling__: xgboost, scikit-learn, GridSearchCV
8. __Evaluation__: scikit-learn, confusion matrix, F1 Score, ROC/AUC

In [7]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

## Data Collection

In [8]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Data Understanding

In [9]:
print('Train shape is:',train.shape)
print('Test shape is:', test.shape)

Train shape is: (42000, 785)
Test shape is: (28000, 784)


In [10]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Data Preparation

#### Get Features and labels

In [12]:
X = train.iloc[:, 1:]

In [13]:
X.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
y = train.iloc[:, 0]

In [15]:
y.head()

0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64

# Modeling

#### Split train set and validation set

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=1)

In [17]:
print("X_train shape is:", X_train.shape)
print("X_val shape is:", X_val.shape)
print("y_train shape is:", y_train.shape)
print("y_val shape is:", y_val.shape)

X_train shape is: (29400, 784)
X_val shape is: (12600, 784)
y_train shape is: (29400,)
y_val shape is: (12600,)


#### xgboost matrix creation

In [18]:
xgb_train = xgb.DMatrix(X_train, label=y_train)
xgb_val   = xgb.DMatrix(X_val, label=y_val)
xgb_test  = xgb.DMatrix(test)

#### specify parameters

In [19]:
params={
'booster':'gbtree',
'objective': 'multi:softmax', #multi classification
'num_class':10, # classes number as this is multi classification
'gamma':0.1,  # prunching parameter, normally , 0.1, 0.2, larger , less prunching
'max_depth':12, # depth of tree, bigger tree will lead to overfitting
'lambda':2,  #  model complexity , L2 regularization, bigger -> less overfitting
'subsample':0.7, # sampling randomly
'colsample_bytree':0.7, # column sampling when tree creation
'min_child_weight':3, 

# ???
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
# smaller, easier overfitting
    
#Minimum sum of instance weight (hessian) needed in a child. 
#If the tree partition step results in a leaf node with the sum of instance weight less than min_child_weight,
#then the building process will give up further partitioning. 
#In linear regression task, this simply corresponds to minimum number of instances needed to be in each node. 
#The larger min_child_weight is, the more conservative the algorithm will be.    
'silent':0 ,# running infor , set to : 0.
'eta': 0.007, # like learning rate
'seed':1000,
'nthread':7,# cpu threads
'tree_method': 'gpu_hist' # Use GPU accelerated algorithm    
#'eval_metric': 'auc'
}


#### specify validations set to watch performance

In [20]:
watchlist = [(xgb_train, 'train'), (xgb_val, 'eval')]

num_iteration = 2000
#num_iteration = 10

#### Train

In [21]:
%%time
model = xgb.train(params, xgb_train, num_boost_round= num_iteration, evals=watchlist, early_stopping_rounds=100)

[0]	train-merror:0.087891	eval-merror:0.124603
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 100 rounds.
[1]	train-merror:0.065646	eval-merror:0.098571
[2]	train-merror:0.058129	eval-merror:0.089841
[3]	train-merror:0.052891	eval-merror:0.080873
[4]	train-merror:0.050408	eval-merror:0.078492
[5]	train-merror:0.047245	eval-merror:0.074762
[6]	train-merror:0.045612	eval-merror:0.073095
[7]	train-merror:0.044184	eval-merror:0.07246
[8]	train-merror:0.043163	eval-merror:0.070317
[9]	train-merror:0.042381	eval-merror:0.070317
[10]	train-merror:0.042041	eval-merror:0.069127
[11]	train-merror:0.041395	eval-merror:0.068889
[12]	train-merror:0.041531	eval-merror:0.068175
[13]	train-merror:0.041088	eval-merror:0.069127
[14]	train-merror:0.04	eval-merror:0.06754
[15]	train-merror:0.039728	eval-merror:0.067063
[16]	train-merror:0.039388	eval-merror:0.067222
[17]	train-merror:0.03966	eval-merror:0.066905
[18]	

[168]	train-merror:0.019592	eval-merror:0.050556
[169]	train-merror:0.01949	eval-merror:0.050476
[170]	train-merror:0.019286	eval-merror:0.050397
[171]	train-merror:0.01932	eval-merror:0.050317
[172]	train-merror:0.019252	eval-merror:0.050397
[173]	train-merror:0.01915	eval-merror:0.050476
[174]	train-merror:0.019116	eval-merror:0.050476
[175]	train-merror:0.018912	eval-merror:0.050317
[176]	train-merror:0.018946	eval-merror:0.050238
[177]	train-merror:0.018878	eval-merror:0.050397
[178]	train-merror:0.018844	eval-merror:0.050317
[179]	train-merror:0.018776	eval-merror:0.050317
[180]	train-merror:0.018707	eval-merror:0.050238
[181]	train-merror:0.018639	eval-merror:0.050238
[182]	train-merror:0.018469	eval-merror:0.050317
[183]	train-merror:0.018333	eval-merror:0.050238
[184]	train-merror:0.018299	eval-merror:0.050079
[185]	train-merror:0.018163	eval-merror:0.050079
[186]	train-merror:0.018197	eval-merror:0.050317
[187]	train-merror:0.018129	eval-merror:0.050397
[188]	train-merror:0.01

[336]	train-merror:0.011565	eval-merror:0.044444
[337]	train-merror:0.011531	eval-merror:0.044365
[338]	train-merror:0.011463	eval-merror:0.044286
[339]	train-merror:0.011429	eval-merror:0.044206
[340]	train-merror:0.011395	eval-merror:0.044048
[341]	train-merror:0.011395	eval-merror:0.044048
[342]	train-merror:0.011361	eval-merror:0.044048
[343]	train-merror:0.011327	eval-merror:0.044048
[344]	train-merror:0.011293	eval-merror:0.044048
[345]	train-merror:0.011259	eval-merror:0.043968
[346]	train-merror:0.011259	eval-merror:0.043968
[347]	train-merror:0.01119	eval-merror:0.043968
[348]	train-merror:0.01119	eval-merror:0.043968
[349]	train-merror:0.011122	eval-merror:0.043889
[350]	train-merror:0.011088	eval-merror:0.043889
[351]	train-merror:0.010986	eval-merror:0.043889
[352]	train-merror:0.010952	eval-merror:0.043968
[353]	train-merror:0.010952	eval-merror:0.043968
[354]	train-merror:0.010952	eval-merror:0.04381
[355]	train-merror:0.010884	eval-merror:0.04381
[356]	train-merror:0.010

[505]	train-merror:0.006156	eval-merror:0.039762
[506]	train-merror:0.006122	eval-merror:0.039683
[507]	train-merror:0.006122	eval-merror:0.039603
[508]	train-merror:0.006088	eval-merror:0.039603
[509]	train-merror:0.006054	eval-merror:0.039524
[510]	train-merror:0.006054	eval-merror:0.039444
[511]	train-merror:0.006054	eval-merror:0.039444
[512]	train-merror:0.006054	eval-merror:0.039444
[513]	train-merror:0.00602	eval-merror:0.039524
[514]	train-merror:0.005952	eval-merror:0.039365
[515]	train-merror:0.005952	eval-merror:0.039524
[516]	train-merror:0.005952	eval-merror:0.039444
[517]	train-merror:0.005884	eval-merror:0.039444
[518]	train-merror:0.005816	eval-merror:0.039365
[519]	train-merror:0.00585	eval-merror:0.039365
[520]	train-merror:0.00585	eval-merror:0.039365
[521]	train-merror:0.005816	eval-merror:0.039444
[522]	train-merror:0.005816	eval-merror:0.039444
[523]	train-merror:0.005782	eval-merror:0.039444
[524]	train-merror:0.005782	eval-merror:0.039444
[525]	train-merror:0.00

[674]	train-merror:0.003537	eval-merror:0.036587
[675]	train-merror:0.003537	eval-merror:0.036587
[676]	train-merror:0.003503	eval-merror:0.036587
[677]	train-merror:0.003503	eval-merror:0.036587
[678]	train-merror:0.003469	eval-merror:0.036587
[679]	train-merror:0.003469	eval-merror:0.036587
[680]	train-merror:0.003469	eval-merror:0.036587
[681]	train-merror:0.003469	eval-merror:0.036587
[682]	train-merror:0.003469	eval-merror:0.036587
[683]	train-merror:0.003469	eval-merror:0.036587
[684]	train-merror:0.003401	eval-merror:0.036587
[685]	train-merror:0.003401	eval-merror:0.036587
[686]	train-merror:0.003367	eval-merror:0.036587
[687]	train-merror:0.003367	eval-merror:0.036508
[688]	train-merror:0.003367	eval-merror:0.036429
[689]	train-merror:0.003333	eval-merror:0.036429
[690]	train-merror:0.003333	eval-merror:0.036508
[691]	train-merror:0.003299	eval-merror:0.036429
[692]	train-merror:0.003299	eval-merror:0.036429
[693]	train-merror:0.003197	eval-merror:0.036349
[694]	train-merror:0

...

[1010]	train-merror:0.000748	eval-merror:0.032381
[1011]	train-merror:0.000748	eval-merror:0.032302
[1012]	train-merror:0.000748	eval-merror:0.032302
[1013]	train-merror:0.000748	eval-merror:0.032381
[1014]	train-merror:0.00068	eval-merror:0.032381
[1015]	train-merror:0.000714	eval-merror:0.032381
[1016]	train-merror:0.00068	eval-merror:0.03246
[1017]	train-merror:0.00068	eval-merror:0.03246
[1018]	train-merror:0.000646	eval-merror:0.032381
[1019]	train-merror:0.000646	eval-merror:0.032381
[1020]	train-merror:0.000646	eval-merror:0.03246
[1021]	train-merror:0.000646	eval-merror:0.03246
[1022]	train-merror:0.000646	eval-merror:0.032381
[1023]	train-merror:0.000646	eval-merror:0.032381
[1024]	train-merror:0.000646	eval-merror:0.032381
[1025]	train-merror:0.000646	eval-merror:0.032302
[1026]	train-merror:0.000646	eval-merror:0.032381
[1027]	train-merror:0.000646	eval-merror:0.032302
[1028]	train-merror:0.000646	eval-merror:0.032381
[1029]	train-merror:0.000646	eval-merror:0.032222
[1030]	

[1176]	train-merror:0.00034	eval-merror:0.031111
[1177]	train-merror:0.00034	eval-merror:0.031111
[1178]	train-merror:0.00034	eval-merror:0.031111
[1179]	train-merror:0.00034	eval-merror:0.03119
[1180]	train-merror:0.00034	eval-merror:0.031111
[1181]	train-merror:0.00034	eval-merror:0.031111
[1182]	train-merror:0.00034	eval-merror:0.031111
[1183]	train-merror:0.00034	eval-merror:0.031111
[1184]	train-merror:0.00034	eval-merror:0.031111
[1185]	train-merror:0.00034	eval-merror:0.031111
[1186]	train-merror:0.00034	eval-merror:0.03119
[1187]	train-merror:0.00034	eval-merror:0.031111
[1188]	train-merror:0.00034	eval-merror:0.031111
[1189]	train-merror:0.00034	eval-merror:0.031111
[1190]	train-merror:0.00034	eval-merror:0.031111
[1191]	train-merror:0.00034	eval-merror:0.031111
[1192]	train-merror:0.00034	eval-merror:0.03119
[1193]	train-merror:0.00034	eval-merror:0.031111
[1194]	train-merror:0.00034	eval-merror:0.031111
[1195]	train-merror:0.00034	eval-merror:0.031111
[1196]	train-merror:0.0

[1342]	train-merror:0.000204	eval-merror:0.029921
[1343]	train-merror:0.000204	eval-merror:0.029921
[1344]	train-merror:0.000204	eval-merror:0.029921
[1345]	train-merror:0.000204	eval-merror:0.029921
[1346]	train-merror:0.000204	eval-merror:0.029921
[1347]	train-merror:0.000204	eval-merror:0.029921
[1348]	train-merror:0.000204	eval-merror:0.029921
[1349]	train-merror:0.000204	eval-merror:0.029921
[1350]	train-merror:0.000204	eval-merror:0.03
[1351]	train-merror:0.000204	eval-merror:0.029841
[1352]	train-merror:0.000204	eval-merror:0.029841
[1353]	train-merror:0.000204	eval-merror:0.029841
[1354]	train-merror:0.000204	eval-merror:0.03
[1355]	train-merror:0.000204	eval-merror:0.03
[1356]	train-merror:0.000204	eval-merror:0.03
[1357]	train-merror:0.000204	eval-merror:0.029921
[1358]	train-merror:0.000204	eval-merror:0.03
[1359]	train-merror:0.000204	eval-merror:0.03
[1360]	train-merror:0.000204	eval-merror:0.03
[1361]	train-merror:0.000204	eval-merror:0.03
[1362]	train-merror:0.000204	eva

[1508]	train-merror:0.000102	eval-merror:0.029444
[1509]	train-merror:0.000102	eval-merror:0.029524
[1510]	train-merror:0.000102	eval-merror:0.029444
[1511]	train-merror:0.000102	eval-merror:0.029365
[1512]	train-merror:0.000102	eval-merror:0.029365
[1513]	train-merror:0.000102	eval-merror:0.029444
[1514]	train-merror:0.000102	eval-merror:0.029444
[1515]	train-merror:0.000102	eval-merror:0.029365
[1516]	train-merror:0.000102	eval-merror:0.029365
[1517]	train-merror:0.000102	eval-merror:0.029365
[1518]	train-merror:0.000102	eval-merror:0.029365
[1519]	train-merror:0.000102	eval-merror:0.029365
[1520]	train-merror:0.000102	eval-merror:0.029365
[1521]	train-merror:0.000102	eval-merror:0.029365
[1522]	train-merror:0.000102	eval-merror:0.029365
[1523]	train-merror:0.000102	eval-merror:0.029286
[1524]	train-merror:0.000102	eval-merror:0.029286
[1525]	train-merror:0.000102	eval-merror:0.029286
[1526]	train-merror:0.000102	eval-merror:0.029286
[1527]	train-merror:0.000102	eval-merror:0.029286


[1675]	train-merror:3.4e-05	eval-merror:0.028968
[1676]	train-merror:3.4e-05	eval-merror:0.028968
[1677]	train-merror:3.4e-05	eval-merror:0.028968
[1678]	train-merror:3.4e-05	eval-merror:0.028968
[1679]	train-merror:3.4e-05	eval-merror:0.028968
[1680]	train-merror:3.4e-05	eval-merror:0.028968
[1681]	train-merror:3.4e-05	eval-merror:0.028968
[1682]	train-merror:3.4e-05	eval-merror:0.028968
[1683]	train-merror:3.4e-05	eval-merror:0.028889
[1684]	train-merror:3.4e-05	eval-merror:0.02881
[1685]	train-merror:3.4e-05	eval-merror:0.02881
[1686]	train-merror:3.4e-05	eval-merror:0.02881
[1687]	train-merror:3.4e-05	eval-merror:0.02873
[1688]	train-merror:3.4e-05	eval-merror:0.02881
[1689]	train-merror:3.4e-05	eval-merror:0.02881
[1690]	train-merror:3.4e-05	eval-merror:0.02881
[1691]	train-merror:3.4e-05	eval-merror:0.02881
[1692]	train-merror:3.4e-05	eval-merror:0.02881
[1693]	train-merror:3.4e-05	eval-merror:0.02881
[1694]	train-merror:3.4e-05	eval-merror:0.02881
[1695]	train-merror:3.4e-05	eva

In [22]:
model.save_model('minist_xgboost.model')
print('Best best_ntree_limit', model.best_ntree_limit)

Best best_ntree_limit 1658


# Evaluation

In [ ]:
%%time
#crash
#preds = model.predict(xgb_test, ntree_limit= model.best_ntree_limit)
#could work
model.set_param('predictor','cpu_predictor')
preds = model.predict(xgb_test)

In [22]:
preds_df = pd.DataFrame(preds)
preds_df['ImageId'] = preds_df.index
preds_df['Label']=preds_df.iloc[:,0].astype(int)
preds_df.drop(0, axis=1, inplace=True)
preds_df.to_csv('xgb_submission2_1.csv', index=False)

In [23]:
bst2 = xgb.Booster(model_file='minist_xgboost.model.backup')

[13:04:12] C:\Users\Administrator\Desktop\xgboost\src\learner.cc:362: Parameter 'predictor' has been recovered from the saved model. It will be set to 'gpu_predictor' for prediction. To override the predictor behavior, explicitly set 'predictor' parameter as follows:
  * Python package: bst.set_param('predictor', [new value])
  * R package:      xgb.parameters(bst) <- list(predictor = [new value])
  * JVM packages:   bst.setParam("predictor", [new value])


####  Crash after setting ntree_limit= model.best_ntree_limit

In [ ]:
preds2 = bst2.predict(xgb_test)

In [25]:
preds_df = pd.DataFrame(preds2)
preds_df['ImageId'] = preds_df.index + 1
preds_df['Label']=preds_df.iloc[:,0].astype(int)
preds_df.drop(0, axis=1, inplace=True)
preds_df.to_csv('xgb_submission2.csv', index=False)